In [1]:
import numpy as np 
import pandas as pd
import statsmodels.api as sm
from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score
from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import cvxopt
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures

In [2]:
def select_train(list_var, data):
    for var in list_var :
        data.pop(var)
    return data

def select_test(list_var, data):
    for var in list_var :
        data.pop(var)
    return data

In [4]:
# 1/ Training data :

column_names = ['Model_name', 'input_shape', 'power', 'input_width', 'input_size', 'nb_params_conv','nb_params_bn', 'nb_params_fc', 'flops', 'nb_layers', 
                'nb_conv_layers', 'nb_bn_layers','nb_fc_layers', 'sum_activations', 'weighted_sum_neurons']
raw_dataset = pd.read_csv('./data/training_dataset.csv', names=column_names,
                      na_values = "?", comment='\t', sep=",", skipinitialspace=True)
train_dataset = raw_dataset.copy()
train_dataset.isna().sum()
train_dataset = train_dataset.dropna()

# 1/ Test data :

column_names = ['Model_name', 'input_shape', 'power', 'input_width', 'input_size', 'nb_params_conv','nb_params_bn', 'nb_params_fc', 'flops', 'nb_layers', 
                'nb_conv_layers', 'nb_bn_layers','nb_fc_layers', 'sum_activations', 'weighted_sum_neurons']
raw_dataset = pd.read_csv('./data/test_dim1.csv', names=column_names,
                      na_values = "?", comment='\t', sep=",", skipinitialspace=True)
test_dataset1 = raw_dataset.copy()
test_dataset1.isna().sum()
test_dataset1 = test_dataset1.dropna()

# 2/ Test data :

column_names = ['Model_name', 'input_shape', 'power', 'input_width', 'input_size', 'nb_params_conv','nb_params_bn', 'nb_params_fc', 'flops', 'nb_layers', 
                'nb_conv_layers', 'nb_bn_layers','nb_fc_layers', 'sum_activations', 'weighted_sum_neurons']
raw_dataset = pd.read_csv('./data/test_dim2.csv', names=column_names,
                      na_values = "?", comment='\t', sep=",", skipinitialspace=True)
test_dataset2 = raw_dataset.copy()
test_dataset2.isna().sum()
test_dataset2 = test_dataset2.dropna()

# 3/ Test data :

column_names = ['Model_name', 'input_shape', 'power', 'input_width', 'input_size', 'nb_params_conv','nb_params_bn', 'nb_params_fc', 'flops', 'nb_layers', 
                'nb_conv_layers', 'nb_bn_layers','nb_fc_layers', 'sum_activations', 'weighted_sum_neurons']
raw_dataset = pd.read_csv('./data/test_dim3.csv', names=column_names,
                      na_values = "?", comment='\t', sep=",", skipinitialspace=True)
test_dataset3 = raw_dataset.copy()
test_dataset3.isna().sum()
test_dataset3 = test_dataset3.dropna()
test_dataset3

,Model_name,input_shape,power,input_width,input_size,nb_params_conv,nb_params_bn,nb_params_fc,flops,nb_layers,nb_conv_layers,nb_bn_layers,nb_fc_layers,sum_activations,weighted_sum_neurons
0,basic_model_75_26,"(75, 75, 3)",5266.0,75,16875,17186368,11616,51300,72648896377,49,26,17,1,17802235,36302064640
1,basic_model_90_11,"(90, 90, 3)",1489.5,90,24300,6221792,4096,501770,11035210512,21,11,4,1,2941792,5509463104
2,basic_model_448_4,"(448, 448, 3)",10510.0,448,602112,12612576,0,72500,570447021917,10,4,0,1,114694168,285116526144
3,basic_model_75_9,"(75, 75, 3)",4427.0,75,16875,5898928,2496,119369800,64936755954,18,9,4,1,12153115,32337687344
4,basic_model_200_17,"(200, 200, 3)",10131.0,200,120000,8992624,1504,29000,426967714982,30,17,6,1,50316448,213448451200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,basic_model_100_47,"(100, 100, 3)",9219.0,100,30000,23035628,13552,49000,459417651813,77,47,24,1,134601231,229628985568
98,basic_model_500_9,"(500, 500, 3)",4765.0,500,750000,727417,4060,16500,39561096708,20,9,6,1,308909190,138440888200
99,basic_model_320_15,"(320, 320, 3)",3588.0,320,307200,1525352,3888,81000,26152797284,30,15,8,1,23250476,13063957568
100,basic_model_480_7,"(480, 480, 3)",9333.0,480,691200,2979364,3472,50100,343520336052,16,7,4,1,221106628,171584275904


In [5]:
list_comp = ['input_width', 'input_size','nb_params_conv', 'nb_params_bn', 'nb_params_fc', 'nb_layers', 
                'nb_conv_layers', 'nb_bn_layers','nb_fc_layers', 'sum_activations', 'weighted_sum_neurons']

list_var = ['input_width', 'nb_params_bn', 'nb_params_fc', 
                'nb_conv_layers', 'nb_bn_layers','nb_fc_layers', 'weighted_sum_neurons']

train_dataset = select_train(list_var, train_dataset)
test_dataset1 = select_test(list_var, test_dataset1)
test_dataset2 = select_test(list_var, test_dataset2)
test_dataset3 = select_test(list_var, test_dataset3)

In [6]:
train_dataset.tail()

,Model_name,input_shape,power,input_size,nb_params_conv,flops,nb_layers,sum_activations
1446,basic_model_320_23,"(1, 320, 320, 3)",7469.0,307200,2499430,168678951282,44,298643104
1447,basic_model_600_42,"(1, 600, 600, 3)",6405.0,1080000,12220341,159725668606,71,140162512
1448,basic_model_320_20,"(1, 320, 320, 3)",10738.0,307200,8674304,1776131430777,37,629248140
1449,basic_model_600_31,"(1, 600, 600, 3)",10586.0,1080000,12481365,853243940125,52,337941414
1450,basic_model_75_12,"(1, 75, 75, 3)",840.0,16875,1179843,3330331266,24,5411083


In [7]:
Y = train_dataset.pop('power')
X = train_dataset

In [8]:
Y_e1 = test_dataset1.pop('power')
X_e1 = test_dataset1

Y_e2 = test_dataset2.pop('power')
X_e2 = test_dataset2

Y_e3 = test_dataset3.pop('power')
X_e3 = test_dataset3

In [9]:
X_train_model_names = X.pop('Model_name')
X_train_input_shapes = X.pop('input_shape')

In [10]:
X_test_model_names1 = X_e1.pop('Model_name')
X_test_input_shapes1 = X_e1.pop('input_shape')

X_test_model_names2 = X_e2.pop('Model_name')
X_test_input_shapes2 = X_e2.pop('input_shape')

X_test_model_names3 = X_e3.pop('Model_name')
X_test_input_shapes3 = X_e3.pop('input_shape')

In [11]:
y_test1 = Y_e1.values
y_test2 = Y_e2.values
y_test3 = Y_e3.values

In [12]:
polyn = PolynomialFeatures(degree=21)
xd = polyn.fit_transform(np.log(X.values))
clf_Ridge = linear_model.Ridge(alpha=0.0001, normalize=True,max_iter=1e5)
A = clf_Ridge.fit(xd,Y.values)

In [13]:
y_pred3 = clf_Ridge.predict(polyn.fit_transform(np.log(X_e3.values)))
y_pred2 = clf_Ridge.predict(polyn.fit_transform(np.log(X_e2.values)))
y_pred1 = clf_Ridge.predict(polyn.fit_transform(np.log(X_e1.values)))

In [23]:
y_test = y_test3
y_pred = y_pred3
test_models = X_test_model_names3.values
test_input_shapes = X_test_input_shapes3.values

In [24]:
print('Explained variance : ',explained_variance_score(y_test, y_pred))
print('R2_value : ',r2_score(y_test, y_pred))
print('Mean absolute error MAE :', mean_absolute_error(y_test, y_pred))
print('Root Mean squared error RMSE :', mean_squared_error(y_test, y_pred, squared=False))
print('Median absolute error MAD:', median_absolute_error(y_test, y_pred))

Explained variance :  0.9676220658468504
R2_value :  0.967490775931587
Mean absolute error MAE : 424.38908906686686
Root Mean squared error RMSE : 591.8992095861678
Median absolute error MAD: 276.39472933063007


In [25]:
# Calculate mean absolute percentage error (MAPE)
errors = abs(y_pred-y_test)
mape = 100 * (errors / y_test) # Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('MAX MAPE: ', round(max(mape), 2), '%.')
print('MAPE:', round(np.mean(mape), 2), '%.')
print('Accuracy:', round(accuracy, 2), '%.')

MAX MAPE:  34.43 %.
MAPE: 7.06 %.
Accuracy: 92.94 %.


In [26]:
import csv
with open('./measured_vs_predicted_Poly_dim3.csv', 'w', newline='') as file : 
    writer = csv.writer(file)
    writer.writerow(['CNN_model', 'input_shape', 'model&shape', 'measured', 'predicted', 'absolute_deviation', 'MAPE (in %)'])
    for i in range(0, len(y_pred)) :
        f = ''+test_models[i]+'_'+test_input_shapes[i]
        writer.writerow([test_models[i], test_input_shapes[i], f, y_test[i], y_pred[i],abs(y_pred[i]-y_test[i]), ((abs(y_pred[i]-y_test[i]))/y_test[i])*100])